In [1]:
from dotenv import load_dotenv
import pandas as pd
import re
import os
import pymssql
import numpy as np
import pyodbc

In [2]:
pd.set_option('display.max_rows',150000)
pd.set_option('display.max_columns',100)
pd.set_option('display.width', 10000)
load_dotenv('.env')
cis_user = os.getenv('CISDBUSER')
pwd = os.getenv('USERPASS')

In [3]:
cis_conn = pymssql.connect(server='SCISPSQLRDS',user=cis_user,password=pwd,database='PROD')
rms_conn = pymssql.connect(server='SPTAPLSQLHPA',port='53130',user=cis_user,password=pwd,database='VMHPALIVE')
cla_conn = pymssql.connect(server='AOEWSPSQLCLRT',port='58427',user=cis_user,password=pwd,database='DMSKPCLARITY')

In [141]:
# cursor = cis_conn.cursor()
# sql1 = ''''''
# df1 = pd.read_sql(sql1, cis_conn)
# cursor.close()

df1 = pd.read_excel('/Users/molinar1/Downloads/MyMSK_MRN_111722.xlsx')
df1.columns = ['Race1ID','RACE','Race2ID','Race2Name','MRN']
df1 = df1[['MRN','Race1ID','RACE','Race2ID','Race2Name']]
df1['MRN'] = df1['MRN'].replace(' ','')
df1['RACE'] = df1['RACE'].str.strip()
df1 = df1[df1['MRN'] != '--------']

In [8]:
cursor = rms_conn.cursor()
sql2 = '''select TOP 10000 * from dbo.ICLMBCTRVISPLNIX'''
df2 = pd.read_sql(sql2, rms_conn)
cursor.close()

In [10]:
df2 = df2.pivot_table(columns=['PLAN_ZZ','BCTR'], index='NUM')
print(df2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [143]:
cursor = cla_conn.cursor()
sql3 = '''select distinct PAT_MRN_ID [MRN],
                r.NAME [RACE]
          from dbo.PATIENT p
          left join dbo.PATIENT_RACE pr
          on p.PAT_ID = pr.PAT_ID
          left join dbo.ZC_PATIENT_RACE r
          on r.PATIENT_RACE_C = pr.PATIENT_RACE_C
          where len(PAT_MRN_ID) = 8
          --and DEATH_DATE is null'''
df3 = pd.read_sql(sql3, cla_conn)
cursor.close()

In [144]:
df4 = pd.read_excel('/Users/molinar1/Downloads/Metriq_Race_110322.xlsx')
df4.columns = ['Metriq_MRN','MRN','RACE']

In [145]:
print(f"Total rows for RMS: {df2.shape[0]}, Total rows for Clarity: {df3.shape[0]}, total rows for MyMSK: {df1.shape[0]}, total rows for Metriq {df4.shape[0]}")

Total rows for RMS: 1776133, Total rows for Clarity: 1745572, total rows for MyMSK: 67617, total rows for Metriq 443640


In [146]:
df = df2.merge(df3, on=['MRN'], suffixes=['_RMS','_CLA'],how='left', indicator=True)
dfa = df2.merge(df3, on=['MRN'], suffixes=['_RMS','_CLA'],how='inner')

dfb = df1.merge(df3, on=['MRN'], suffixes=['_MyMSK','_CLA'],how='left', indicator=True)
dfc = df1.merge(df2, on=['MRN'], suffixes=['_MyMSK','_RMS'], how='left', indicator=True)

dfd = df4.merge(df2, on=['MRN'], suffixes=['_RMS','_Metriq'], how='left', indicator=True)

dfs = [dfc,dfd]

In [147]:
for f in dfs:
  race_match = []
  match = []
  for col in f.columns:
    if 'RACE' in col:
      match.append(col)

  for idx, row in f.iterrows():
    if str(row[match[0]]) == str(row[match[1]]).replace('-',' '):
      race_match.append('Match')
    else:
      race_match.append('No Match')

  f['RACE_MATCH'] = race_match

RACE_MyMSK RACE_RMS
67617
RACE_RMS RACE_Metriq
443640


In [148]:
print(f"{dfb[dfb['_merge'].isin(['both'])].reset_index().drop('index', axis=1).shape[0]} out of {dfb.shape[0]} are a match, {round(dfb[dfb['_merge'].isin(['both'])].reset_index().drop('index', axis=1).shape[0]/dfb.shape[0]*100,2)}%, {dfb[dfb['_merge'].isin(['left_only'])].reset_index().drop('index', axis=1).shape[0]} MRNs not matched between Clarity and MyMSK")

print(f"{dfc[dfc['_merge'].isin(['both'])].reset_index().drop('index', axis=1).shape[0]} out of {dfc.shape[0]} are a match, {round(dfc[dfc['_merge'].isin(['both'])].reset_index().drop('index', axis=1).shape[0]/dfc.shape[0]*100,2)}%, {dfc[dfc['_merge'].isin(['left_only'])].reset_index().drop('index', axis=1).shape[0]} MRNs not matched between RMS and MyMSK")

print(f"{dfd[dfd['_merge'].isin(['both'])].reset_index().drop('index', axis=1).shape[0]} out of {dfd.shape[0]} are a match, {round(dfd[dfd['_merge'].isin(['both'])].reset_index().drop('index', axis=1).shape[0]/dfd.shape[0]*100,2)}%, {dfd[dfd['_merge'].isin(['left_only'])].reset_index().drop('index', axis=1).shape[0]} MRNs not matched between RMS and Metriq")

print(f"{df[df['_merge'].isin(['both'])].reset_index().drop('index', axis=1).shape[0]} out of {df.shape[0]} are a match, {round(df[df['_merge'].isin(['both'])].reset_index().drop('index', axis=1).shape[0]/df.shape[0]*100,2)}%, {df[df['_merge'].isin(['left_only'])].reset_index().drop('index', axis=1).shape[0]} MRNs not matched between Clarity and RMS")

67572 out of 67617 are a match, 99.93%, 45 MRNs not matched between Clarity and MyMSK
67598 out of 67617 are a match, 99.97%, 19 MRNs not matched between RMS and MyMSK
443631 out of 443640 are a match, 100.0%, 9 MRNs not matched between RMS and Metriq
1745279 out of 1776143 are a match, 98.26%, 30864 MRNs not matched between Clarity and RMS


In [ ]:
dfb1 = dfb[dfb['_merge'].isin(['left_only'])].reset_index().drop('index', axis=1)
dfc1 = dfc[dfc['_merge'].isin(['left_only'])].reset_index().drop('index', axis=1)
dfd1 = dfd[dfd['_merge'].isin(['left_only'])].reset_index().drop('index', axis=1)
file = 'MRN-Race-nomatch.xlsx'
writer = pd.ExcelWriter(file, engine='xlsxwriter')
dfb1.to_excel(writer, index=False, sheet_name='Clarity-MyMSK-NoMatch')
dfc1.to_excel(writer, index=False, sheet_name='RMS-MyMSK-NoMatch')
dfd1.to_excel(writer, index=False, sheet_name='RMS-Metriq-NoMatch')
writer.save()

In [150]:
dfb1 = dfb[dfb['_merge'].isin(['both'])].reset_index().drop('index', axis=1)
dfc1 = dfc[dfc['_merge'].isin(['both'])].reset_index().drop('index', axis=1)
dfd1 = dfd[dfd['_merge'].isin(['both'])].reset_index().drop('index', axis=1)

file = 'MRN-Race-Match.xlsx'
writer = pd.ExcelWriter(file, engine='xlsxwriter')
dfb1.to_excel(writer, index=False, sheet_name='Clarity-MyMSK-Match')
dfc1.to_excel(writer, index=False, sheet_name='RMS-MyMSK-Match')
dfd1.to_excel(writer, index=False, sheet_name='RMS-Metriq-Match')
writer.save()

In [ ]:
half = round(df.shape[0]/2)
halfa = round(dfa.shape[0]/2)

file = 'MRN-RACE-Compare.xlsx'
writer = pd.ExcelWriter(file, engine='xlsxwriter')

df.iloc[:half,:].to_excel(writer, index=False, sheet_name='MRN_Pt1')
df.iloc[half:,:].to_excel(writer, index=False, sheet_name='MRN_Pt2')
dfa.iloc[:halfa,:].to_excel(writer, index=False, sheet_name='Matches_1')
dfa.iloc[halfa:,:].to_excel(writer, index=False, sheet_name='Matches_2')
writer.save()